# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. 

In [ ]:
!pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package

If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells. 

In [4]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas as pd #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [ ]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [ ]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [ ]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [ ]:
 dir(wikiContentRequest)

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [ ]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])

This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [ ]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [ ]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)
contentPTags[0].text

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [ ]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
(contentParagraphsDF)

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [ ]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')


In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [ ]:
print(regexResults.group(0))

That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [ ]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

In [ ]:
re.sub(r'\d+', '', 'egemen(1234) pamukcu &123 deneme *12* $99 23 bitti')

Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [ ]:
joe_biden = 'https://www.rev.com/blog/transcripts/joe-biden-speech-transcript-on-covid-19-economic-recovery-plan'
biden_request = requests.get(joe_biden) #requesting a transcript of Joe Biden's speech on COVID

In [ ]:
biden_soup = bs4.BeautifulSoup(biden_request.text, 'html.parser')

In [ ]:
biden_p = biden_soup.find_all('p') #looking at source, speech content seems to be tagged as <p>

In [ ]:
for i in range(6):
    print('\n\nparagraph{}\n'.format(i), biden_p[i])

#seems like the speech content starts at paragraph5
#paragraph0 is the date of speech
#paragraph1 is brief info on speech
#paragraph2, paragraph3, and paragraph4 are irrelevant 
#paragrap5 is where the speech begins

In [ ]:
biden_p[-2:] #final paragraph seems to be irrelevant as well

In [ ]:
#creating a pandas DataFrame with following columns
import pandas as pd
biden_df = pd.DataFrame(columns=['Paragraph No', 'Speech Date', 'Speaker',
                                 'Speech Title', 'Speech Content', 'Speech Summary'])

In [ ]:
#I will count the paragraphs starting with the paragraph5 and not include the final
#paragraph in the DataFrame, so I will eliminate 6 paragraphs in total
p_no = []
content = []
speaker = []

#I can use the head tag in html as the title of the speech, might be useful
title = [re.sub('\n', '', biden_soup.head.text)] * len(biden_p[5:-1])

#I will also take paragraph0 and paragraph1 as date and summary of speech, they can be useful too
date = [biden_p[0].text] * len(biden_p[5:-1])
info = [biden_p[1].text] * len(biden_p[5:-1])

for i, p in enumerate(biden_p[5:-1]):
    p_no.append(i)
    
    #using regular expressions to eliminate clutter
    text = re.sub(r'\(\d+\:\d+\)\n', '',p.text)
    
    #in these speech paragraphs, strings before first ':' character is the speaker name
    #variable 'speak' identifies the index where speaker name ends and speech content begins
    speak = re.search(':', text).start()
    speaker.append(text[:speak])
    
    #I can remove speaker name from speech text
    text = text[speak + 1:]
    content.append(text)

#assigning lists to columns in the DataFrame
biden_df['Paragraph No'] = p_no
biden_df['Speech Date'] = date
biden_df['Speech Summary'] = info
biden_df['Speech Content'] = content
biden_df['Speaker'] = speaker
biden_df['Speech Title'] = title

biden_df


# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [ ]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

In [ ]:
print(contentPTags)

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [ ]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [ ]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [ ]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF


# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [76]:
def getTextFromURLs(urls):
    '''
    Takes a list of urls and returns a DataFrame object.
    '''
    
    df_main = pd.DataFrame(columns=['Paragraph No', 'Speech Date', 'Speaker',
                                    'Speech Title', 'Speech Content', 'Speech Summary'])
    for url in urls:
        req = requests.get(url)
        soup = bs4.BeautifulSoup(req.text, 'html.parser')
        p = soup.find_all('p')
        df = pd.DataFrame(columns=['Paragraph No', 'Speech Date', 'Speaker',
                                 'Speech Title', 'Speech Content', 'Speech Summary'])
        p_no = []
        content = []
        speaker = []
        title = [re.sub('\n', '', soup.head.text)] * len(p[5:-1])
        date = [p[0].text] * len(p[5:-1])
        info = [p[1].text] * len(p[5:-1])
        for i, par in enumerate(p[5:-1]):
            p_no.append(i)
            text = re.sub(r'\(\d+\:\d+\)\n', '',par.text)
            text = re.sub(r'\[\w+ \d+\:\d+\:\d+\] ', '', text) #to eliminate text like "[inaudible 00:04:25]"
            if re.search(':', text) != None:
                speak = re.search(':', text).start()
                speaker.append(text[:speak])
                text = text[speak + 1:]
            else:
                speaker.append(None)
            content.append(text)
            
        
        df['Paragraph No'] = p_no
        df['Speech Date'] = date
        df['Speech Summary'] = info
        df['Speech Content'] = content
        df['Speaker'] = speaker
        df['Speech Title'] = title
                
        df_main = pd.concat([df_main, df], axis=0)
        
    return df_main.reset_index(drop=True)

In [77]:
def spiderSpeeches(source):
    '''
    This function takes a URL of a website containing a speech transcript from rev.com
    and returns a Pandas DataFrame including content from original URL and URLs linked
    from the source speech. 
    '''
    allURLs = [source]
    req = requests.get(source)
    soup = bs4.BeautifulSoup(req.text, 'html.parser')
    for related in soup.find('div', id='related-transcripts').find_all('a'):
        allURLs.append(related['href'])
    return getTextFromURLs(allURLs)

In [57]:
spiderSpeeches('https://www.rev.com/blog/transcripts/donald-trump-mike-pence-rally-speech-transcript-traverse-city-michigan-november-2')

,Paragraph No,Speech Date,Speaker,Speech Title,Speech Content,Speech Summary
0,0,"Nov 2, 2020",Mike Pence,Donald Trump & Mike Pence Rally Speech Transcr...,That’s what I’m talking about. Thank you for ...,Donald Trump and Mike Pence held a campaign ra...
1,1,"Nov 2, 2020",Mike Pence,Donald Trump & Mike Pence Rally Speech Transcr...,It’s on Michigan. And the road to victory goe...,Donald Trump and Mike Pence held a campaign ra...
2,2,"Nov 2, 2020",Mike Pence,Donald Trump & Mike Pence Rally Speech Transcr...,And let me say what a privilege it is to be h...,Donald Trump and Mike Pence held a campaign ra...
3,3,"Nov 2, 2020",Mike Pence,Donald Trump & Mike Pence Rally Speech Transcr...,"But in three short years, we rebuilt our mili...",Donald Trump and Mike Pence held a campaign ra...
4,4,"Nov 2, 2020",Mike Pence,Donald Trump & Mike Pence Rally Speech Transcr...,The President said it was the biggest quarter...,Donald Trump and Mike Pence held a campaign ra...
...,...,...,...,...,...,...
205,14,"Jan 14, 2021",Rep. Steny Hoyer,Steny Hoyer Closing Argument on Trump 2nd Impe...,"This is about our country, our Constitution, ...",Representative Steny Hoyer gave the Democrats’...
206,15,"Jan 14, 2021",Rep. Steny Hoyer,Steny Hoyer Closing Argument on Trump 2nd Impe...,"Do not pretend my friends, that it was simply...",Representative Steny Hoyer gave the Democrats’...
207,16,"Jan 14, 2021",Rep. Steny Hoyer,Steny Hoyer Closing Argument on Trump 2nd Impe...,"It was just days ago that the President, afte...",Representative Steny Hoyer gave the Democrats’...
208,17,"Jan 14, 2021",Rep. Steny Hoyer,Steny Hoyer Closing Argument on Trump 2nd Impe...,"We do this today, not for politics. We don’t ...",Representative Steny Hoyer gave the Democrats’...


### RECURSIVE SEGMENT

In [21]:
import copy
jane_elliott = 'https://www.rev.com/blog/transcripts/jane-elliott-speech-transcript-on-race-being-black-in-america'

In [22]:
def returnURLs(source):
    '''
    Takes a source URL as input and returns list of referenced URLs including the source.
    '''
    req = requests.get(source)
    soup = bs4.BeautifulSoup(req.text, 'html.parser')
    allURLs = [source]
    for related in soup.find('div', id='related-transcripts').find_all('a'):
        allURLs.append(related['href'])
    return allURLs

In [23]:
def spider_r(source, depth, allURLs):
    '''
    Recursive function that takes depth and source URL as parameters and
    returns list of URLs based on the depth input.
    '''
    if depth == 0:
        allURLs += returnURLs(source)
        return allURLs
    
    else:
        lst = returnURLs(source)
        allURLs += lst
        newURLs = copy.deepcopy(allURLs)
        for url in lst:
            if url != source:
                newURLs += spider_r(url, depth - 1, allURLs)
        return list(set(newURLs))

In [24]:
def spider_helper(source, depth):
    '''
    Helper function that calls spider_r with an empty list as allURLs parameter.
    '''
    return spider_r(source, depth, [])

In [25]:
a = spider_helper(jane_elliott, 0)
b = spider_helper(jane_elliott, 1)
c = spider_helper(jane_elliott, 2)

print(len(a), len(b), len(c))
#numbers of URLs do not increase exponentially because most of the pages reference 
#each other and the function returns unique URLs only. 

4 8 12


In [87]:
press = 'https://www.rev.com/blog/transcripts/ohio-gov-mike-dewine-press-conference-on-inauguration-preparedness-transcript-january-14'
d = spider_helper(press, 3)
len(d)

5

In [85]:
pd.set_option('max_rows', None)
getTextFromURLs(d)

,Paragraph No,Speech Date,Speaker,Speech Title,Speech Content,Speech Summary
0,0,"Jan 15, 2021",Speaker 1,DOJ Update on Capitol Attackers January 15: Tr...,(silence) Hello?. The call will begin shortly...,"The Department of Justice held a briefing, upd..."
1,1,"Jan 15, 2021",Speaker 1,DOJ Update on Capitol Attackers January 15: Tr...,Thank you for holding for the US Department o...,"The Department of Justice held a briefing, upd..."
2,2,"Jan 15, 2021",Speaker 2,DOJ Update on Capitol Attackers January 15: Tr...,Good day and welcome to the Department of Jus...,"The Department of Justice held a briefing, upd..."
3,3,"Jan 15, 2021",Mark Ramone,DOJ Update on Capitol Attackers January 15: Tr...,Thank you. And thank you all for joining us. ...,"The Department of Justice held a briefing, upd..."
4,4,"Jan 15, 2021",Mark Ramone,DOJ Update on Capitol Attackers January 15: Tr...,"And then we have an individual, Ashan Benedic...","The Department of Justice held a briefing, upd..."
5,5,"Jan 15, 2021",Michael Sherwin,DOJ Update on Capitol Attackers January 15: Tr...,"Okay. So, hello everyone. It’s Mike Sherwin h...","The Department of Justice held a briefing, upd..."
6,6,"Jan 15, 2021",Michael Sherwin,DOJ Update on Capitol Attackers January 15: Tr...,"As related to those 275 open investigations, ...","The Department of Justice held a briefing, upd..."
7,7,"Jan 15, 2021",Michael Sherwin,DOJ Update on Capitol Attackers January 15: Tr...,"But as the investigation continues, as the da...","The Department of Justice held a briefing, upd..."
8,8,"Jan 15, 2021",Michael Sherwin,DOJ Update on Capitol Attackers January 15: Tr...,"And this was the individual, I think that’s r...","The Department of Justice held a briefing, upd..."
9,9,"Jan 15, 2021",Michael Sherwin,DOJ Update on Capitol Attackers January 15: Tr...,"And I think as we said earlier, we don’t care...","The Department of Justice held a briefing, upd..."


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [92]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

#print(r.text[:1000])
print(r.text[-100:])


de193a0b4b00c1c7a9b857e2812964e.jpg","photos":[],"tags":["cat","cats","lol","lolcat","lolcats"]}]};



This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [93]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [100]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [104]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))
            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,639732477227892736,https://64.media.tumblr.com/31e8882b2fd934f7b9...,"Fri, 08 Jan 2021 10:00:32","[cat, cats, lol, lolcat, lolcats]",jpg
1,639724919901585408,https://64.media.tumblr.com/fb868dea41853ad734...,"Fri, 08 Jan 2021 08:00:25","[cat, cats, lol, lolcat, lolcats]",jpg
2,639717378220015616,https://64.media.tumblr.com/61e3d59b36423880a9...,"Fri, 08 Jan 2021 06:00:33","[cat, cats, lol, lolcat, lolcats]",jpg
3,639709877361688576,https://64.media.tumblr.com/12f20ceaaa85f5a91e...,"Fri, 08 Jan 2021 04:01:19","[cat, cats, lol, lolcat, lolcats]",jpg
4,639702313021440000,https://64.media.tumblr.com/2386aff8793f75dcd7...,"Fri, 08 Jan 2021 02:01:06","[cat, cats, lol, lolcat, lolcats]",jpg
5,639687179569889280,https://64.media.tumblr.com/61e3d59b36423880a9...,"Thu, 07 Jan 2021 22:00:33","[cat, cats, lol, lolcat, lolcats]",jpg
6,639679639402479617,https://64.media.tumblr.com/61e3d59b36423880a9...,"Thu, 07 Jan 2021 20:00:42","[cat, cats, lol, lolcat, lolcats]",jpg
7,638101724326936576,https://64.media.tumblr.com/653968fb91a4356937...,"Mon, 21 Dec 2020 10:00:25","[cat, cats, lol, lolcat, lolcats]",png
8,637014579763871744,https://64.media.tumblr.com/e649f37db75d7901b9...,"Wed, 09 Dec 2020 10:00:43","[cat, cats, lol, lolcat, lolcats]",png
9,637007022068498432,https://64.media.tumblr.com/9e5288c65670863e89...,"Wed, 09 Dec 2020 08:00:36","[cat, cats, lol, lolcat, lolcats]",jpg


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [106]:
example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [107]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [108]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [109]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [110]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
julius_caesar.txt,"Dramatis Personae\n\n JULIUS CAESAR, Roman st..."
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
tempest.txt,"The Tempest\n\nActus primus, Scena prima.\n\nA..."
phoenix_and_the_turtle.txt,THE PHOENIX AND THE TURTLE\n\nby William Shake...
king_lear.txt,The Tragedie of King Lear\n\n\nActus Primus. S...
passionate_pilgrim.txt,THE PASSIONATE PILGRIM\n\nby William Shakespea...
cymbeline.txt,The Tragedie of Cymbeline\n\nActus Primus. Sco...
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...
two_gentlemen_of_verona.txt,THE TWO GENTLEMEN OF VERONA\n\nby William Shak...
rape_of_lucrece.txt,THE RAPE OF LUCRECE\n\nby William Shakespeare\...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [114]:
information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.5
%����
97 0 obj
<<
/Length 2988      
/Filter /FlateDecode
>>
stream
xڝYY��~�_��T,U�����f<{����z\[)o(
���!�������n��4�J�/b�� }7�9�s������o���I�,�|�a�d�J��M��y�9?�~<i]כHE���	�}����e��k��esp�ߋ�շyS�Jf��|�?>���׺�+��y7����==w��8�����M<�x�
߉�M�Q�k�k�tW�C�6덂
������拍���Y��o乤�8s���Fy-�����g��vGvM ����3���}���[+o�y��שnٰ|7N6UŽ�f����l����D�D��|[i�n��MY��f�3��W�n`�>���X!~lڧ��>��K�7�
��P�z������s;
�zx�����t��|ӎ���n���V{�Av@�Wt:�!bc!*���>����΋�@R�Q`��_�A���Aл�yG+J���z[��\[G�1-r��U�m�A�|��'�2


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [115]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [120]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

In [134]:
type(infoExtractionBytes)

_io.BytesIO

Now we can give it to pdfminer.

In [133]:
print(readPDF(infoExtractionBytes)[:550])

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determini


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [138]:
example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [139]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [140]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

name 'example_docx_save' is not defined


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [142]:
d = docx.Document(downloadIfNeeded(example_docx, 'deneme.docx', mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [5]:
IMDbComments = []
IDs = []
for file in os.scandir('neg'):
    with open(file.path) as f:
        IMDbComments.append(re.sub(r'\<\w+ \/\>', ' ', f.read()))
    IDs.append(file.name)

In [6]:
urls = []
with open('urls_neg.txt') as u:
    for line in u:
        urls.append(line)

In [7]:
comments_df = pd.DataFrame({'IDs': IDs, 'IMDb Comments': IMDbComments, 'URLs': urls})
comments_df

,IDs,IMDb Comments,URLs
0,1821_4.txt,Working with one of the best Shakespeare sourc...,http://www.imdb.com/title/tt0064354/usercommen...
1,10402_1.txt,"Well...tremors I, the original started off in ...",http://www.imdb.com/title/tt0100680/usercommen...
2,1062_4.txt,Ouch! This one was a bit painful to sit throug...,http://www.imdb.com/title/tt0100680/usercommen...
3,9056_1.txt,"I've seen some crappy movies in my life, but t...",http://www.imdb.com/title/tt0100680/usercommen...
4,5392_3.txt,"""Carriers"" follows the exploits of two guys an...",http://www.imdb.com/title/tt0047200/usercommen...
...,...,...,...
12495,3307_3.txt,"My comments may be a bit of a spoiler, for wha...",http://www.imdb.com/title/tt0306063/usercommen...
12496,2240_4.txt,"The ""saucy"" misadventures of four au pairs who...",http://www.imdb.com/title/tt0306063/usercommen...
12497,11556_1.txt,"Oh, those Italians! Assuming that movies about...",http://www.imdb.com/title/tt0034455/usercommen...
12498,4617_3.txt,Eight academy nominations? It's beyond belief....,http://www.imdb.com/title/tt0406816/usercommen...


### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

